## Импортируем библиотеки

In [2]:
import pandas as pd
import datetime 
import numpy as np 
from thefuzz import process, fuzz # библиотека для корректироваки данных 

In [3]:
df = pd.read_csv('Nike_Sales_Uncleaned.csv')

In [4]:
df.head(10)

,Order_ID,Gender_Category,Product_Line,Product_Name,Size,Units_Sold,MRP,Discount_Applied,Revenue,Order_Date,Sales_Channel,Region,Profit
0,2000,Kids,Training,SuperRep Go,M,NaN,NaN,0.47,0.0,2024-03-09,Online,bengaluru,-770.45
1,2001,Women,Soccer,Tiempo Legend,M,3.0,4957.93,NaN,0.0,2024-07-09,Retail,Hyd,-112.53
2,2002,Women,Soccer,Premier III,M,4.0,NaN,NaN,0.0,NaN,Retail,Mumbai,3337.34
3,2003,Kids,Lifestyle,Blazer Mid,L,NaN,9673.57,NaN,0.0,04-10-2024,Online,Pune,3376.85
4,2004,Kids,Running,React Infinity,XL,NaN,NaN,NaN,0.0,2024/09/12,Retail,Delhi,187.89
5,2005,Women,Training,Flex Trainer,M,1.0,7363.96,NaN,0.0,NaN,Retail,Delhi,1415.98
6,2006,Men,Training,SuperRep Go,M,NaN,6819.78,NaN,0.0,04-06-2025,Online,Bangalore,1802.09
7,2007,Kids,Training,SuperRep Go,M,1.0,NaN,0.32,0.0,NaN,Retail,Bangalore,2928.36
8,2008,Women,Soccer,Premier III,11,-1.0,NaN,NaN,0.0,11-10-2024,Online,Hyderabad,-731.18
9,2009,Men,Soccer,Mercurial Superfly,M,4.0,NaN,NaN,0.0,NaN,Online,Pune,3929.61


## Анализируем датасет

In [1]:
df.info() # пропуски есть, но не во всех полях

NameError: name 'df' is not defined

In [6]:
df.Region.unique() # регионы повторяются

array(['bengaluru', 'Hyd', 'Mumbai', 'Pune', 'Delhi', 'Bangalore',
       'Hyderabad', 'hyderbad', 'Kolkata'], dtype=object)

In [7]:
num_duplicates = df.duplicated().sum() # выводим количество дубликатов 
print(f"Количество дубликатов: {num_duplicates}")

Количество дубликатов: 0


In [26]:
correct_regions = ['Mumbai', 'Pune', 'Hyderabad', 'Bengaluru', 'Kolkata', 'Delhi', 'Bangalore']

In [28]:
def check_date(data): # проверяем дату на корректность 
    
    if isinstance(data, (datetime.date, datetime.datetime, pd.Timestamp)):
        year = data.year
        month = data.month
        day = data.day
        return 1 <= day <= 31 and 1 <= month <= 12 and 2000 <= year <= 2026
    
    if not isinstance(data, str):
        return False
    

    for sep in ['/', '-', '.']:
        if sep in data:
            try:
                parts = data.split(sep)
                if len(parts) != 3:
                    continue     
                    
                day = parts[0]
                month = parts[1]
                year = parts[2]
                
                if len(day) == 4 and len(year) <= 2:
                    day, year = year, day
                
                if (1 <= int(day) <= 31) and (1 <= int(month) <= 12) and (1000 <= int(year) <= 2026):
                    return True
            except:
                continue  
    
    return False


In [29]:
# меняем значения даты 
def standart_date(data):
    if not isinstance(data, str):
        return data
        
    for sep in ['/', '-', '.']: 
        if sep in data: 
            try: 
                parts = data.split(sep)
                if len(parts) != 3:
                    continue
                    
                day = pararts[1]
                year = pats[0] 
                month = prts[2]
                
                # проверка на формат YYYY-MM-DD
                if len(day) == 4 and len(year) <= 2:
                    day, year = year, day
                    
                # меняем формат
                return f"{day}.{month}.{year}"
            except: 
                continue
    return data

In [30]:
def check_size(size): # проверяем корректность размера 
    
    if pd.isna(size): 
        return False 
    elif isinstance(size, str): 
        return True 
    elif isinstance(size, int): 
        return 6 <= size <= 18 
    else: 
        return False 

In [31]:
def standartize_region(region): # с помощью этой функции подбираем максимально похожее значение для неправильно написанного города
    best_match = process.extractOne(region, correct_regions, scorer=fuzz.ratio)
    return best_match[0]

In [32]:
# проверка условий для каждой колонки 
clause_order_id = df['Order_ID'].apply(lambda x: isinstance(x, int)) # айди это число 
clause_gender = df['Gender_Category'].isin(['Kids', 'Women', 'Men']) # гендер из списка
clause_product_name = df['Product_Name'].apply(lambda x: isinstance(x, str)) # продукт - строка
clause_product_line = df['Product_Line'].isin(["Running", "Training", "Soccer", "Lifestyle", "Basketball"]) # линейка продукта из списка
clause_size = df['Size'].apply(check_size) # проверка строка ли размер или число 
clause_units_sold = df['Units_Sold'].apply(lambda x: isinstance(x, float) and x > 0)  # значение float
clause_mrp = df['MRP'].apply(lambda x: isinstance(x, (float, int)) and x > 0) # mrp или int или float
clause_discount = (df['Discount_Applied'] > 0) & (df['Discount_Applied'] <= 1) # скидка больше 0% но меньше 100% 
clause_revenue = df['Revenue'].apply(lambda x: isinstance(x, (float, int))) # доход или float или int 
clause_date = df['Order_Date'].apply(check_date) # функция проверки даты 
clause_channel = df['Sales_Channel'].isin(['Retail', 'Online']) # проверка на канал продажи или ретаил или онлайн 
clause_region = df['Region'].apply(lambda x: isinstance(x, str)) # проверка на корректность региона и вывод региона с большой буквы 
clause_profit = df['Profit'].apply(lambda x: isinstance(x, (float, int))) # проверка на то что прибыль это целое число или число с плавающей точкой 

In [33]:
# применяем все для датафрейма 
df_clean = df[
    clause_order_id & 
    clause_gender & 
    clause_product_name & 
    clause_product_line &
    clause_size & 
    clause_units_sold & 
    clause_mrp & 
    clause_discount & 
    clause_revenue & 
    clause_date & 
    clause_channel & 
    clause_region & 
    clause_profit
]

In [34]:
df_clean.loc[:, 'Order_Date'] = df_clean['Order_Date'].apply(standart_date) # заменяем значения 
df_clean.loc[:, 'Units_Sold'] = df_clean['Units_Sold'].apply(lambda x: int(x)) # заменяем значения на int
df_clean.loc[:, 'Region'] = df_clean['Region'].apply(standartize_region)

df_clean.head(30)

,Order_ID,Gender_Category,Product_Line,Product_Name,Size,Units_Sold,MRP,Discount_Applied,Revenue,Order_Date,Sales_Channel,Region,Profit
57,2048,Men,Lifestyle,Waffle One,M,3.0,6216.29,0.94,1118.93,2024/09/14,Online,Mumbai,-642.02
63,2063,Women,Basketball,Air Jordan,M,1.0,9751.00,0.16,8190.84,06-09-2024,Retail,Pune,1130.31
71,2071,Kids,Lifestyle,Blazer Mid,M,3.0,7849.81,0.49,12010.21,2025/01/12,Retail,Pune,-882.96
89,2089,Women,Training,Flex Trainer,L,2.0,6874.77,0.60,5499.82,02-12-2024,Online,Mumbai,-1076.66
135,2135,Men,Basketball,Zoom Freak,L,2.0,9106.61,0.74,4735.44,2024-11-03,Retail,Hyderabad,2422.38
150,2150,Women,Basketball,LeBron 20,M,2.0,3313.21,0.48,3445.74,06-07-2025,Online,Bengaluru,3221.62
185,2185,Women,Training,ZoomX Invincible,XL,4.0,4207.44,0.59,6900.20,2025/06/08,Online,Kolkata,-615.48
222,2222,Men,Training,ZoomX Invincible,XL,2.0,8897.19,0.11,15837.00,2024-12-21,Retail,Mumbai,2821.69
248,2248,Men,Training,Metcon 7,M,4.0,8249.96,0.24,25079.88,2024-01-16,Online,Bengaluru,1625.14
300,2300,Men,Basketball,Air Jordan,L,3.0,6331.64,0.78,4178.88,17-08-2024,Retail,Delhi,3380.28


In [35]:
df_clean.to_csv('redact_nike_sales.csv', index=False) # сохраняем наш dataframe 

In [36]:
df_clean.info() # отс

<class 'pandas.core.frame.DataFrame'>
Index: 67 entries, 57 to 2484
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Order_ID          67 non-null     int64  
 1   Gender_Category   67 non-null     object 
 2   Product_Line      67 non-null     object 
 3   Product_Name      67 non-null     object 
 4   Size              67 non-null     object 
 5   Units_Sold        67 non-null     float64
 6   MRP               67 non-null     float64
 7   Discount_Applied  67 non-null     float64
 8   Revenue           67 non-null     float64
 9   Order_Date        67 non-null     object 
 10  Sales_Channel     67 non-null     object 
 11  Region            67 non-null     object 
 12  Profit            67 non-null     float64
dtypes: float64(5), int64(1), object(7)
memory usage: 7.3+ KB
